In [ ]:
# default_exp ctx

# CTX
> Utils for working with MRO CTX data

In [ ]:
# hide
from nbdev.showdoc import show_doc

In [ ]:
# export

from subprocess import CalledProcessError

import hvplot.xarray
import xarray as xr
from dask import compute, delayed
from kalasiris.pysis import (
    ProcessError,
    ctxcal,
    ctxevenodd,
    getkey,
    mroctx2isis,
    spiceinit,
)
from planetarypy.config import config
from planetarypy.pds.apps import get_index
from planetarypy.utils import file_variations, url_retrieve
from tqdm.auto import tqdm
from yarl import URL

baseurl = URL(
    "https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/"
)
storage_root = config.storage_root / "mro/ctx"
edrindex = get_index("mro.ctx.indexes.edr")

In [ ]:
# export 
def catch_isis_error(func):
    def inner(*args, **kwargs):
        try:
            func(*args, **kwargs)
        except ProcessError as err:
            print('Had ISIS error:')
            print(' '.join(err.cmd))
            print(err.stdout)
            print(err.stderr)
    return inner

In [ ]:
# export


class CTXEDR:
    storage = storage_root / "edr"

    def __init__(self, id_):
        """Class to manage CTX data products."""
        self.product_id = id_
        (self.cub_name, self.cal_name, self.destripe_name) = file_variations(
            self.local_path, [".cub", ".cal.cub", ".dst.cal.cub"]
        )
        self.is_read=False

    @property
    def product_id(self):
        return self._product_id

    @product_id.setter
    def product_id(self, value):
        self.is_read=False
        self._product_id = value

    @property
    def id(self):
        "for laziness"
        return self.product_id

    @property
    def meta(self):
        s = edrindex.query("PRODUCT_ID == @self.id").squeeze()
        s.index = s.index.str.lower()
        return s

    @property
    def local_folder(self):
        return self.storage / self.meta.volume_id / self.id

    @property
    def local_path(self):
        return self.local_folder / f"{self.id}.IMG"

    @property
    def url(self):
        "Calculate URL from input dataframe row."
        url = baseurl / self.meta.volume_id.lower() / "data" / (self.id + ".IMG")
        return url

    def download(self, overwrite=False):
        self.local_folder.mkdir(parents=True, exist_ok=True)
        if self.local_path.exists() and not overwrite:
            print("File exists. Use `overwrite=True` to download fresh.")
            return
        url_retrieve(self.url, self.local_path)

    @catch_isis_error
    def isis_import(self):
        mroctx2isis(from_=self.local_path, to=self.cub_name)

    @catch_isis_error
    def spice_init(self):
        spiceinit(from_=self.cub_name, web="yes")

    @catch_isis_error
    def calibrate(self):
        ctxcal(from_=self.cub_name, to=self.cal_name)
        self.is_calib_read = False

    @catch_isis_error
    def destripe(self):
        if self.do_destripe():
            ctxevenodd(from_=self.cal_name, to=self.destripe_name)
            self.destripe_name.rename(self.cal_name)

    @catch_isis_error
    def do_destripe(self):
        value = int(
            getkey(
                from_=self.cal_name,
                objname="isiscube",
                grpname="instrument",
                keyword="SpatialSumming",
            )
        )
        return False if value == 2 else True

    def calib_pipeline(self, overwrite=False):
        if self.cal_name.exists() and not overwrite:
            return
        pbar = tqdm("isis_import spice_init calibrate destripe".split())
        for name in pbar:
            pbar.set_description(name)
            getattr(self, name)()
        pbar.set_description("Done.")
            
    def read_edr(self):
        "`da` stands for dataarray, standard abbr. within xarray."
        if not self.is_read:
            self.edr_da = xr.open_rasterio(self.local_path)
            self.is_read = True
        return self.edr_da

    def read_calibrated(self):
        "`da` stands for dataarray, standard abbr. within xarray."
        if not self.is_calib_read:
            self.cal_da = xr.open_rasterio(self.cal_name)
            self.is_calibd_read = True
        return self.cal_da

    def plot_da(self, da):
        return da.isel(band=0, drop=True).hvplot(
            x="y", y="x", rasterize=True, cmap="gray", data_aspect=1
        )

    def plot_calibrated(self):
        return self.plot_da(self.read_calibrated())
    
    def __str__(self):
        s = f"PRODUCT_ID: {self.product_id}\n"
        s += f"URL: {self.url}\n"
        s += f"Local: {self.local_path}\n"
        s += f"Shape: {self.read_edr().shape}"
        return s

    def __repr__(self):
        return self.__str__()

In [ ]:
id_ = 'F10_039666_1383_XN_41S315W'

In [ ]:
ctx = CTXEDR(id_)

In [ ]:
ctx

/home/maye/miniconda3/envs/py38/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


PRODUCT_ID: F10_039666_1383_XN_41S315W
URL: https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2337/data/F10_039666_1383_XN_41S315W.IMG
Local: /home/maye/big_drive/planetary_data/mro/ctx/edr/MROX_2337/F10_039666_1383_XN_41S315W/F10_039666_1383_XN_41S315W.IMG
Shape: (1, 15360, 5056)

In [ ]:
ctx.id

'F10_039666_1383_XN_41S315W'

In [ ]:
assert ctx.id == ctx.product_id

In [ ]:
ctx.meta

volume_id                                                 MROX_2337
file_specification_name         DATA/F10_039666_1383_XN_41S315W.IMG
original_product_id                                4A_04_10C800EF00
product_id                               F10_039666_1383_XN_41S315W
image_time                               2015-01-12 06:36:38.896000
instrument_id                                                   CTX
instrument_mode_id                                             NIFL
line_samples                                                   5056
lines                                                         15360
spatial_summing                                                   1
scaled_pixel_width                                             5.04
pixel_aspect_ratio                                              1.2
emission_angle                                                 1.29
incidence_angle                                                41.1
phase_angle                                     

In [ ]:
ctx.storage

Path('/home/maye/big_drive/planetary_data/mro/ctx/edr')

In [ ]:
ctx.local_folder

Path('/home/maye/big_drive/planetary_data/mro/ctx/edr/MROX_2337/F10_039666_1383_XN_41S315W')

In [ ]:
ctx.url

URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2337/data/F10_039666_1383_XN_41S315W.IMG')

In [ ]:
ctx.download()

File exists. Use `overwrite=True` to download fresh.


In [ ]:
ctx.isis_import()

In [ ]:
ctx.spice_init()

In [ ]:
ctx.calibrate()

In [ ]:
ctx.destripe()

In [ ]:
ctx.calib_pipeline()

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
ctx.plot_calibrated()

:DynamicMap   []
   :Image   [y,x]   (value)

In [ ]:
ds = ctx.read_edr()
ds

<xarray.DataArray (band: 1, y: 15360, x: 5056)>
[77660160 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 0.5 1.5 2.5 3.5 ... 1.536e+04 1.536e+04 1.536e+04
  * x        (x) float64 0.5 1.5 2.5 3.5 ... 5.054e+03 5.054e+03 5.056e+03
Attributes:
    transform:   (1.0, 0.0, 0.0, 0.0, 1.0, 0.0)
    res:         (1.0, -1.0)
    is_tiled:    0
    nodatavals:  (0.0,)
    scales:      (1.0,)
    offsets:     (0.0,)

In [ ]:
# export

class CTXEDRCollection:
    """Class to deal with a set of CTX products."""
    def __init__(self, product_ids):
        self.product_ids = product_ids

    def get_urls(self):
        """Get URLs for list of product_ids.
                
        Returns
        -------
        List[yarl.URL]
            List of URL objects with the respective PDS URL for download.
        """
        urls = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            ctx.product_id = p_id
            urls.append(ctx.url)
        self.urls = urls
        return urls
    
    def download_collection(self):
        lazys = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            lazys.append(delayed(ctx.download)())
        print("Launching parallel download...")
        compute(*lazys)
        print("Done.")
        
    def calibrate_collection(self):
        lazys = []
        for p_id in self.product_ids:
            ctx = CTXEDR(p_id)
            lazys.append(delayed(ctx.calib_pipeline)())
        print("Launching parallel calibration...")
        compute(*lazys)
        print("Done.")
        
    def calib_exist_check(self):
        return [(p_id, CTXEDR(p_id).cal_name.exists()) for p_id in self.product_ids]

In [ ]:
ids = edrindex.sample(5).PRODUCT_ID
ids

97393    K03_054576_1256_XN_54S219W
72784    F13_040898_0948_XI_85S075W
26143    B17_016392_1436_XI_36S321W
91098    J14_050165_1356_XI_44S032W
59159    D19_034650_1494_XN_30S187W
Name: PRODUCT_ID, dtype: object

In [ ]:
coll = CTXEDRCollection(ids)

In [ ]:
coll.get_urls()

[URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_3050/data/K03_054576_1256_XN_54S219W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2375/data/F13_040898_0948_XI_85S075W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0952/data/B17_016392_1436_XI_36S321W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_2883/data/J14_050165_1356_XI_44S032W.IMG'),
 URL('https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_1941/data/D19_034650_1494_XN_30S187W.IMG')]

In [ ]:
lazys = coll.download_collection()
lazys

Launching parallel download...


J14_050165_1356_XI_44S032W.IMG
:   0%|          | 0/264049600 [00:00<?, ?it/s]

B17_016392_1436_XI_36S321W.IMG
:   0%|          | 0/238162880 [00:00<?, ?it/s]

D19_034650_1494_XN_30S187W.IMG
:   0%|          | 0/36246464 [00:00<?, ?it/s]

K03_054576_1256_XN_54S219W.IMG
:   0%|          | 0/54364640 [00:00<?, ?it/s]

F13_040898_0948_XI_85S075W.IMG
:   0%|          | 0/165680064 [00:00<?, ?it/s]

Done.


In [ ]:
def process_fname(fname, get_name_only=False, keep_all=False, overwrite=False):
    obsid = os.path.splitext(fname)[0]
    (cub_name, cal_name, destripe) = file_variations(
        obsid, [".cub", ".cal.cub", ".dst.cal.cub"]
    )


class CTXCalibrator:
    def __init__(self, product_id):
        self.product_id = product_id
        (self.cub_name, self.cal_name, self.destripe_name) = file_variations(
            product_id, [".cub", ".cal.cub", ".dst.cal.cub"]
        )
        
    def isis_import(self):
        

In [ ]:
ctxcal = CTXCalibrator(ctx.product_id)

In [ ]:
ctxcal.cub_name

Path('F10_039666_1383_XN_41S315W.cub')

In [ ]:
    mroctx2isis(from_=fname, to=cub_name)
    print('spice init')
    stdout.flush()
    spiceinit(from_=cub_name)

    print('Radiometric Calibration of CTX Data')
    stdout.flush()
    ctxcal(from_=cub_name, to=cal_name)

    # doing this to keep it functional with Meg's older pysis
    if hasattr(getkey, 'check_output'):
        mygetkey = getkey.check_output
    else:
        mygetkey = getkey

    value = int(mygetkey(from_=cal_name,
                         objname="isiscube",
                         grpname="instrument",
                         keyword="SpatialSumming"))

    if (value != 2):
        print("Destriping...")
        ctxevenodd(from_=cal_name, to=destripe)
        # to keep things consistent for the search for push-able CTX files:
        print("Renaming destriped to [...].cal.cub")
        os.rename(destripe, cal_name)

    if not keep_all:
        os.remove(cub_name)

    return cal_name